# Overview

This notebook uses the 'TextFeaturesGenerator' class (from text_features) to convert textual data into qunatitaive data. 

For now, it creates a bag-of-words representation and a tf-idf representation. We will also add SVD/PCA of these matrices and a Word2Vec representation in the next few days.

Will update the TextFeaturesGenerator class on an ongoing basis and update the usage here.

In [1]:
from text_features import TextFeaturesGenerator
from project_helper import TweetData
import pandas as pd
import numpy as np
from datetime import timedelta  
import datetime

Reusing the TweetData class to get cleaned tweets.

In [2]:
tweet_data = TweetData()
tweet_data.clean_tweets.head()

,tweets,timestamp,after4_date
timestamp,,,
2019-10-02 23:41:51-05:00,democrats want to steal the election,2019-10-02 23:41:51-05:00,2019-10-03
2019-10-02 23:27:52-05:00,mississippi there is a very important election...,2019-10-02 23:27:52-05:00,2019-10-03
2019-10-02 23:27:52-05:00,he loves our military and supports our vets de...,2019-10-02 23:27:52-05:00,2019-10-03
2019-10-02 21:06:36-05:00,look at this photograph,2019-10-02 21:06:36-05:00,2019-10-03
2019-10-02 19:51:56-05:00,schiff house intel chairman got early account ...,2019-10-02 19:51:56-05:00,2019-10-03


# Daily Tweets

This does the following two things:

1) Change the date of the tweets after 3 PM Chicago time to the following day (as trading closes then)
2) Concatenate all tweets in a given day to one large document

In [3]:
tweet_data.daily_tweets.head()

,tweets
date,
2009-05-05,donald trump will be appearing on the view tom...
2009-05-08,donald trump reads top ten financial tips on l...
2009-05-09,new blog post celebrity apprentice finale and ...
2009-05-12,my persona will never be that of a wallflower ...
2009-05-13,miss usa tara conner will not be fired ive alw...


Creating a 'TextFeaturesGenerator' instance which takes the tweets as an argument

In [3]:
feature_generator = TextFeaturesGenerator(tweet_data.clean_tweets.tweets)

'get_bow_matrix' creates the bag-of-words matrix

In [4]:
bow_mat = feature_generator.get_bow_matrix()

In [5]:
bow_mat.shape

(27960, 17359)

The shape of this matrix is 27.96K rows (same number as the tweets) and the columns are 16,781, which is equal to the unique number of words in the vocabulary.

In [6]:
bow_mat[:10,:10].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

As you can see, most of the values are zero which is why it is stored as a 'sparse-matrix'

Bag-of-words is simply a count of words in the tweet. A better representation is 'tf-idf'. The 'get_tfidf_matrix' creates

In [7]:
tfidf_mat = feature_generator.get_tfidf_matrix()
tfidf_mat.shape

(27960, 17359)

The matrices can be saved using the matrices function. You can either specify a 'folder' which will be created and both matrices stored in it, else will store in the working directory.

In [8]:
feature_generator.save_matrices()

The two matrices will be saved with the names "bow_mat.npz" and "tfidf_mat.npz"

You can also specify a folder and a suffix to the file names.

In [9]:
feature_generator.save_matrices(folder="matrices",suffix="_v2")

The files can be loaded using the following commands:

In [10]:
from scipy import sparse
bow_loaded = sparse.load_npz("bow_mat.npz")
tfidf_loaded = sparse.load_npz("tfidf_mat.npz")
print(bow_loaded.shape)
print(tfidf_loaded.shape)

(27960, 17359)
(27960, 17359)


## PCA (through SVD) of the matrices

You can get the SVD of the bow and tfidf matrices as well.

In [11]:
svd_bow_mat = feature_generator.get_svd_bow_mat()

In [12]:
svd_bow_mat.shape

(27960, 2)

By default, it gives back two components. You can changet that using the n_components argument.

In [13]:
svd_bow_mat = feature_generator.get_svd_bow_mat(n_components=100)

In [14]:
svd_bow_mat.shape

(27960, 100)

You can get the SVD of the tf-idf as well.

In [15]:
svd_tfidf_mat = feature_generator.get_svd_bow_mat(n_components=100)

In [16]:
svd_tfidf_mat.shape

(27960, 100)

These matrices can be saved as well.

In [17]:
feature_generator.save_matrices()

You can load them back using np.load

In [18]:
svd_loaded_mat = np.load('svd_tfidf_mat.npy')

In [19]:
svd_loaded_mat.shape

(27960, 100)

# Aggregagte SVD per day 

In [20]:
svd_df = pd.DataFrame(svd_loaded_mat)

In [21]:
svd_df['timestamp'] = tweet_data.clean_tweets.index
svd_df['date'] = svd_df.timestamp.dt.date

In [22]:
svd_df.head()

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,timestamp,date
0,1.069162,0.073472,-0.803908,-0.202682,-0.503965,0.011639,0.038896,-0.052699,-0.081935,-0.020411,...,-0.166631,0.021926,-0.253401,-0.222502,0.198627,0.008263,-0.152623,0.299796,2019-10-02 23:41:51-05:00,2019-10-02
1,2.399010,-2.640690,1.537751,0.988793,0.253774,-0.426361,-1.617954,0.713951,-1.783920,0.935433,...,-0.032958,0.415026,0.126267,-0.180378,-0.130619,-0.139637,0.166141,0.111495,2019-10-02 23:27:52-05:00,2019-10-02
2,2.036368,-2.172903,2.635054,0.428314,-0.241706,-0.040651,-0.673243,-0.356856,-1.043026,1.124688,...,0.153755,-0.073918,0.482440,-0.324832,0.101042,-0.633148,-0.093890,0.105424,2019-10-02 23:27:52-05:00,2019-10-02
3,0.116249,-0.056975,0.003445,0.020397,0.103324,-0.119210,-0.130229,-0.093681,-0.024051,-0.028973,...,-0.014578,0.099306,0.025237,-0.080506,0.028129,-0.063209,0.024518,0.057879,2019-10-02 21:06:36-05:00,2019-10-02
4,0.474283,-0.025551,0.216644,0.714577,0.521690,0.847947,-0.057162,-0.040227,0.004715,-0.261764,...,-0.182929,0.091129,0.048752,-0.103810,0.100549,0.045351,0.108905,-0.143518,2019-10-02 19:51:56-05:00,2019-10-02


In [23]:
svd_df_daily = svd_df.groupby('date').agg(np.mean)

In [24]:
svd_df_daily

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
date,,,,,,,,,,,,,,,,,,,,,
2009-05-04,1.917539,-0.741860,-0.006851,-0.307680,0.114204,-0.790058,0.078518,-0.845050,-0.826081,-0.329043,...,0.107552,0.112172,0.038968,-0.187356,0.075312,0.159345,-0.025975,0.042474,-0.014691,-0.026927
2009-05-05,1.736160,-0.734847,-0.035298,-0.524848,-0.118777,-0.626884,-0.976174,-0.785034,-0.511813,-0.418262,...,-0.016422,-0.051565,-0.047186,-0.051693,0.096850,0.125929,0.047916,0.083566,-0.094729,0.003240
2009-05-08,0.657092,0.022901,0.337051,-0.141766,-0.178049,-0.143132,-0.152205,-0.137561,-0.320182,-0.021014,...,-0.042821,-0.014263,0.085546,-0.055233,-0.002863,0.027041,-0.044515,0.024860,-0.011708,-0.030250
2009-05-12,0.763673,-0.616437,-0.249637,-0.133147,0.902717,-0.351933,-0.332288,-0.596769,0.318566,-0.321151,...,-0.085345,-0.071135,0.003974,-0.007599,0.333226,0.070844,-0.114315,0.026641,0.025812,0.231355
2009-05-13,0.552410,-0.719471,-0.641173,-0.030115,-0.049287,0.127540,0.068206,-0.100774,-0.274819,-0.080949,...,-0.091908,-0.072269,0.051072,-0.010656,0.045475,0.100131,0.017376,0.052313,-0.048777,-0.057950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-28,1.764919,-0.019437,-0.006130,-0.071961,-0.089839,-0.060733,0.146705,0.280572,-0.044637,0.081915,...,0.008993,0.062977,0.012227,0.071581,-0.042681,0.002053,0.006528,0.064066,-0.013356,-0.004471
2019-09-29,1.811718,0.070875,-0.140102,-0.027907,-0.079576,-0.108192,0.294700,-0.024339,0.161656,0.077195,...,-0.014578,0.006286,-0.006692,0.002615,-0.057323,-0.021273,0.093232,0.015066,0.035226,-0.000386
2019-09-30,2.817178,0.213592,0.116190,0.067874,-0.165713,-0.020973,0.172849,0.026442,0.002949,-0.039970,...,0.049921,-0.037052,-0.004556,0.029124,-0.044566,-0.007839,0.014728,-0.017043,0.013898,0.005306


In [25]:
svd_df_daily.to_csv('svd_df_daily.csv')

# 4 PM

In [26]:
tweet_data.clean_tweets['timestamp'] = tweet_data.clean_tweets.index
after_4_tweets = tweet_data.clean_tweets.timestamp.dt.hour >= 15
tweet_data.clean_tweets['after4_date'] = tweet_data.clean_tweets.timestamp.dt.date
tweet_data.clean_tweets.loc[after_4_tweets,'after4_date'] = tweet_data.clean_tweets.timestamp[after_4_tweets].dt.date + timedelta(days=1)

In [27]:
tweet_data.clean_tweets.head(100)

,tweets,timestamp,after4_date
timestamp,,,
2019-10-02 23:41:51-05:00,democrats want to steal the election,2019-10-02 23:41:51-05:00,2019-10-03
2019-10-02 23:27:52-05:00,mississippi there is a very important election...,2019-10-02 23:27:52-05:00,2019-10-03
2019-10-02 23:27:52-05:00,he loves our military and supports our vets de...,2019-10-02 23:27:52-05:00,2019-10-03
2019-10-02 21:06:36-05:00,look at this photograph,2019-10-02 21:06:36-05:00,2019-10-03
2019-10-02 19:51:56-05:00,schiff house intel chairman got early account ...,2019-10-02 19:51:56-05:00,2019-10-03
...,...,...,...
2019-09-28 03:55:17-05:00,thank you to general mcmaster just more fake n...,2019-09-28 03:55:17-05:00,2019-09-28
2019-09-28 02:03:00-05:00,,2019-09-28 02:03:00-05:00,2019-09-28
2019-09-27 19:41:18-05:00,i am draining the swamp,2019-09-27 19:41:18-05:00,2019-09-28


In [28]:
combined_daily_tweets = tweet_data.clean_tweets.groupby('after4_date')['tweets'].apply(lambda x: ' '.join(x))
combined_daily_tweets.head()

after4_date
2009-05-05    donald trump will be appearing on the view tom...
2009-05-08    donald trump reads top ten financial tips on l...
2009-05-09    new blog post celebrity apprentice finale and ...
2009-05-12    my persona will never be that of a wallflower ...
2009-05-13    miss usa tara conner will not be fired ive alw...
Name: tweets, dtype: object

In [29]:
combined_daily_tweets.to_csv('combined_daily_tweets.csv')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


# Check if the concatenation is correct

In [30]:
tweet_data.clean_tweets.tweets[tweet_data.clean_tweets.after4_date==pd.to_datetime("2019-10-03")]

timestamp
2019-10-02 23:41:51-05:00                democrats want to steal the election 
2019-10-02 23:27:52-05:00    mississippi there is a very important election...
2019-10-02 23:27:52-05:00    he loves our military and supports our vets de...
2019-10-02 21:06:36-05:00                             look at this photograph 
2019-10-02 19:51:56-05:00    schiff house intel chairman got early account ...
2019-10-02 15:48:47-05:00    the do nothing democrats should be focused on ...
2019-10-02 15:39:07-05:00    adam schiff should only be so lucky to have th...
2019-10-02 15:31:53-05:00    democrats are trying to undo the election rega...
2019-10-02 15:31:03-05:00    nancy pelosi just said that she is interested ...
2019-10-02 15:19:09-05:00    all of this impeachment nonsense which is goin...
2019-10-02 15:02:11-05:00    now the press is trying to sell the fact that ...
Name: tweets, dtype: object

In [31]:
combined_daily_tweets[combined_daily_tweets.index.values==pd.to_datetime("2019-10-03")]

after4_date
2019-10-03    democrats want to steal the election  mississi...
Name: tweets, dtype: object

# Create SVD matrix of the combiened 4 PM tweets

In [32]:
combined_generator = TextFeaturesGenerator(combined_daily_tweets)

In [33]:
n_components = 2
combined_svd_df = pd.DataFrame(combined_generator.get_svd_tfidf_mat(n_components=n_components))

In [34]:
combined_svd_df['after4_date'] = combined_daily_tweets.index.values

In [35]:
combined_svd_df.head()

,0,1,after4_date
0,0.231291,0.194568,2009-05-05
1,0.052782,0.062221,2009-05-08
2,0.079785,0.035236,2009-05-09
3,0.098248,0.041978,2009-05-12
4,0.069910,0.062147,2009-05-13


In [36]:
combined_svd_df.to_csv('combined_svd_df.csv')